In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install sentencepiece

In [ ]:
import numpy as np
import pandas as pd
!pip install datasets
!pip install sacrebleu
!pip insall accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "insall" - maybe you meant "install"


In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

import torch
from torch.utils.data import random_split
import datasets
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
from datasets import load_metric
import gc
import torch
from transformers import AutoTokenizer
import datasets

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/custom_train_3_1to21.csv')
train.head()

,Sentence,English_Translation
0,@someUSER congratulations on you celebrating b...,@some users congratulate you for celebrating B...
1,@LoKarDi_RT uske liye toh bahot kuch karna pad...,"@Lokardi_ rat we should a lot more for that, b..."
2,@slimswamy yehi to hum semjhane ki koshish kar...,"@Slimswami ehi, this is what i'm expecting you..."
3,@DramebaazKudi cake kaha hai ??,@Where is Dramebajakudi where is the cake?
4,@someUSER i'm in hawaii at the moment . home ...,@some user Don't want to come home next friday...


In [ ]:
val = pd.read_csv('/content/drive/MyDrive/st2.csv')
val = val.rename(columns = {'Hinglish' : 'Sentence', 'English' : 'English_Translation'})
val.head()

,Sentence,English_Translation
0,Abe bigg boss show me baki contestants bhi hai...,Now there are other contestants in the Bigg Bo...
1,Filhal toh behanchod zindagi rape kr rae hai.,At present the behanchod life is raping.
2,agar tumhare sath rape ho jaye to tumhare fath...,What will your father do if Rape happens to you?
3,"I hate it, hostel me banti thi. Us din chawal ...",I hate it. was made in hostel. that time i was...
4,"Tumhare to terrorism failane aate Hain, Indian...","You come to spread terrorism, the Indian Army ..."


In [ ]:
train = pd.concat([train, val])
train.shape

(25558, 2)

In [ ]:
train['Sentence'].info()

<class 'pandas.core.series.Series'>
Int64Index: 25558 entries, 0 to 499
Series name: Sentence
Non-Null Count  Dtype 
--------------  ----- 
25558 non-null  object
dtypes: object(1)
memory usage: 399.3+ KB


In [ ]:
train['English_Translation'].info()

<class 'pandas.core.series.Series'>
Int64Index: 25558 entries, 0 to 499
Series name: English_Translation
Non-Null Count  Dtype 
--------------  ----- 
25558 non-null  object
dtypes: object(1)
memory usage: 399.3+ KB


In [ ]:
train.dropna(inplace = True)

In [ ]:
train.shape, val.shape#, test.shape

((25558, 2), (500, 2))

In [ ]:
train_rows = train
val_rows = val

In [ ]:
def load_val():
    d = []
    for i in range(0, val_rows.shape[0]):
      src = val_rows.iloc[i, :]['Sentence']
      tgt = val_rows.iloc[i, :]['English_Translation']
      print("SRC: ", src)
      print("TRG: ", tgt)
      d.append(
              {
                  "translation": {
                      "hinglish": src.strip(),
                      "english": tgt.strip()
                  }
              }
      )
    print(f'total size of data is {len(d)}')
    tdata = pd.DataFrame(d)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    return dataset

val_set = load_val()

SRC:  Abe bigg boss show me baki contestants bhi hai un se related news deta nhi...sirf sari updates hina se related deta hai...wo bhi hate wali...khabari hai bakri chabar chabar karta rehta hina k bare me
TRG:  Now there are other contestants in the Bigg Boss show and you not giving news related to them. why all updates related to hina? that too hatted... khabri is chatting always about hina.
SRC:  Filhal toh behanchod zindagi rape kr rae hai.
TRG:  At present the behanchod life is raping.
SRC:  agar tumhare sath rape ho jaye to tumhare father kya karenge
TRG:  What will your father do if Rape happens to you?
SRC:  I hate it, hostel me banti thi. Us din chawal me ghee aur namak daal ke khati thi
TRG:  I hate it. was made in hostel. that time i was eating by mixing salt and butter in the rice.
SRC:  Tumhare to terrorism failane aate Hain, Indian army 72 hooron se mila deti unko
TRG:  You come to spread terrorism, the Indian Army would get meet 72 Hurons to them.

SRC:  100 Dino mai Boh

In [ ]:
def load_train():
    d = []
    for i in range(0, train_rows.shape[0]):
      src = train_rows.iloc[i, :]['Sentence']
      tgt = train_rows.iloc[i, :]['English_Translation']
      print("SRC: ", src)
      print("TRG: ", tgt)
      d.append(
              {
                  "translation": {
                      "hinglish": src.strip(),
                      "english": tgt.strip()
                  }
              }
      )
    print(f'total size of data is {len(d)}')
    tdata = pd.DataFrame(d)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    return dataset

train_set = load_train()

Streaming output truncated to the last 5000 lines.
You don't know about fact as yu don't know about the scientific methods
SRC:  tum raints ho 
TRG:  
You are rants
SRC:  aap ek mdhysthta pr bolte hain, jhan aap utne hi khali sind ke sath sbse apriy hain jaise aap apne moorkh sonnet aur apne bojhil gle ko pas krte haina apni ajnjanta se khush aur mshmallow ko pas krte hain 
TRG:   You speak on an arbitration where you are most unwelcome with the same empty sind as you pass your idiotic sonnet and your fat ass: happy of your ignorancy and passing the marshmallow
SRC:  iske lie main is prstav ko smrthn krta hoon. 
TRG:  

 

For what it is worth, I support this motion
SRC:  ekseel ne chrcha men kuchh yogdan nhin diya hai, Igor ya rbj ya ybm jaise any adhikansh upyogkrtaon ke viprit, aur aksr baten uljhata hai 
TRG:   XAL has contributed nothing to the discussion, in stark contraindistinction to most of the other users like Igor or rbj or ybm, and often stirs things up
SRC:  mauroo talk k

In [ ]:
val_set

Dataset({
    features: ['id', 'translation'],
    num_rows: 500
})

In [ ]:
train_set

Dataset({
    features: ['id', 'translation'],
    num_rows: 25558
})

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[250004, 35378, 4, 903, 1632, 149357, 38, 2], [250004, 3293, 83, 15700, 149357, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "hinglish"
target_lang = "english"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    # no need this line
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train_set = train_set.map(preprocess_function, batched=True)
tokenized_val_set = val_set.map(preprocess_function, batched=True)
#tokenized_test_set = test_set.map(preprocess_function, batched=True)

Map:   0%|          | 0/25558 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_set['translation']

[{'english': "@some users congratulate you for celebrating British kid singers Sophia Grace's and Rosie's 1st anniversary visit of your show",
  'hinglish': "@someUSER congratulations on you celebrating british kid singers sophia grace's and rosie's 1st anniversary of a visit of your show .  how"},
 {'english': '@Lokardi_ rat we should a lot more for that, by this evi people nothing will happen #ForTheSakeOfHumanity',
  'hinglish': '@LoKarDi_RT uske liye toh bahot kuch karna padega ye pappiyon se kaam nahi chalega #ForTheSakeOfHumanity'},
 {'english': "@Slimswami ehi, this is what i'm expecting you to understand, people invest everything in this isnt it.",
  'hinglish': '@slimswamy yehi to hum semjhane ki koshish kar rahe hain. Log to sab kuch ko issi mein tol dete hain. Context:http://www.opindia.com/2015/02/buzzfeed-editor-tries-to-make-political-tweets-you-wont-believe-what-happened-next/\xa0…'},
 {'english': '@Where is Dramebajakudi where is the cake?',
  'hinglish': '@DramebaazKud

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
metric = load_metric("sacrebleu")

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

In [ ]:
args = Seq2SeqTrainingArguments(output_dir="./mbart_810/",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=2,
                        per_device_eval_batch_size=2,
                        learning_rate=5e-5,
                        num_train_epochs=2,
                        predict_with_generate=True,
                        logging_dir="/logs",
                        logging_steps=10000,
                        save_steps=10000,
                        report_to="none"
                        )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in __init__:116                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1333 in __post_init__      │
│                                                                                                  │
│   1330 │   │   elif not isinstance(self.report_to, list):                                        │
│   1331 │   │   │   self.report_to = [self.report_to]                                             │
│   1332 │   │                                                                                     │
│ ❱ 1333 │   │   if self.warmup_ratio < 0 or self.warmup_ratio > 1:                                │
│   1334 │   │   │   raise ValueError("warmup_ratio must lie in range [0,1]")                      │
│   1335 │   │   elif self.warmup_ratio > 0 and self.warmup_steps > 0:                             │
│   1336 │   │   │   logger.info(                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1697 in device             │
│                                                                                                  │
│   1694 │   │   return self._setup_devices                                                        │
│   1695 │                                                                                         │
│   1696 │   @property                                                                             │
│ ❱ 1697 │   def n_gpu(self):                                                                      │
│   1698 │   │   """                                                                               │
│   1699 │   │   The number of GPUs used by this process.                                          │
│   1700                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:54 in __get__              │
│                                                                                                  │
│    51 │   │   attr = "__cached_" + self.fget.__name__                                            │
│    52 │   │   cached = getattr(obj, attr, None)                                                  │
│    53 │   │   if cached is None:                                                                 │
│ ❱  54 │   │   │   cached = self.fget(obj)                                                        │
│    55 │   │   │   setattr(obj, attr, cached)                                                     │
│    56 │   │   return cached                                                                      │
│    57                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1631 in _setup_devices     │
│                                                                                                  │
│   1628 │   │   │   # workaround for setups like notebooks where the launcher can't be used,      │
│   1629 │   │   │   # but deepspeed requires a dist env.                                          │
│   1630 │   │   │   # env LOCAL_RANK could be set manually by the user, or via init_distributed   │
│ ❱ 1631 │   │   │   self.local_rank = int(os.environ.get("LO

In [ ]:
trainer = Seq2SeqTrainer(model=model,
                args=args,
                data_collator=data_collator,
                train_dataset=tokenized_train_set,
                eval_dataset=tokenized_val_set,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
                )

tokenized_datasets, tokenized_test_set, trainer2 = accelerator.prepare(
     tokenized_train_set, tokenized_val_set, trainer
      )

trainer.train()

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/test_st2.csv')
test = test.rename(columns = {'Hinglish' : 'Sentence', 'English' : 'English_Translation'})
test.head()

In [ ]:
test_rows = test

In [ ]:
def load_test():
    d = []
    for i in range(0, test_rows.shape[0]):
      src = test_rows.iloc[i, :]['Sentence']
      tgt = test_rows.iloc[i, :]['English_Translation']
      print("SRC: ", src)
      print("TRG: ", tgt)
      d.append(
              {
                  "translation": {
                      "hinglish": src.strip(),
                      "english": tgt.strip()
                  }
              }
      )
    print(f'total size of data is {len(d)}')
    tdata = pd.DataFrame(d)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    return dataset

test_set = load_test()

In [ ]:
test_set

In [ ]:
tokenized_test_set = test_set.map(preprocess_function, batched=True)
tokenized_test_set = accelerator.prepare(tokenized_test_set)

In [ ]:
predictions = trainer.predict(tokenized_test_set)

In [ ]:
predictions.predictions

In [ ]:
import pickle as pkl

In [ ]:
with open('/content/drive/MyDrive/801_test_predictions_list_3.pkl', 'wb') as f:
  pkl.dump(predictions.predictions, f)

In [ ]:
predictions.label_ids

In [ ]:
predictions.metrics

In [ ]:
tokenizer

In [ ]:
tokenizer.decode(predictions.predictions[0])

In [ ]:
for i in range(0, len(predictions.predictions)):
  print("Test data point number ", i)
  print("Source English: ")
  print(tokenized_test_set[i]['translation']['english'])
  print("Source Hinglish: ")
  print(tokenized_test_set[i]['translation']['hinglish'])
  print("Translation: ")
  print(tokenizer.decode(predictions.predictions[i]))
  print()

In [ ]:
x_test = []
y_test = []
y_pred = []
for i in range(0, len(predictions.predictions)):
  x_test.append(tokenized_test_set[i]['translation']['english'])
  y_test.append(tokenized_test_set[i]['translation']['hinglish'])
  y_pred.append(tokenizer.decode(predictions.predictions[i]))

In [ ]:
len(x_test), len(y_test), len(y_pred)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
bleu = []
# reference and predicted sentences
for i in range(0, 1000):
  ref_sent = y_test[i].split()
  pred_sent = y_pred[i].split()
  # calculate BLEU score
  score = sentence_bleu(ref_sent, pred_sent)
  bleu.append(score)

print(bleu)

In [ ]:
import sacrebleu
ref = ['This is a test .']
hyp = ['This is a train .']
bleu = sacrebleu.corpus_bleu(hyp, [ref])
print(bleu.score)

In [ ]:
predictions.predictions

In [ ]:
preds = []
for i in range(0, len(predictions.predictions)):
  d = tokenizer.decode(predictions.predictions[i])
  preds.append(d)

In [ ]:
x_test = []
for i in range(0, len(tokenized_test_set)):
  d = tokenized_test_set[i]['translation']['english']
  x_test.append(d)

In [ ]:
y_test = []
for i in range(0, len(tokenized_test_set)):
  d = tokenized_test_set[i]['translation']['hinglish']
  y_test.append(d)

In [ ]:
x_test

In [ ]:
preds

In [ ]:
y_test

In [ ]:
x_test

In [ ]:
preds

In [ ]:
import pickle
with open("preds_3.pkl", "wb") as f:
    # Use the pickle.dump() method to pickle the data to the file
    pickle.dump(preds, f)

In [ ]:
import pickle
with open("y_test_3.pkl", "wb") as f:
    pickle.dump(y_test, f)

In [ ]:
import pickle
with open("x_test_3.pkl", "wb") as f:
    pickle.dump(x_test, f)

In [ ]:
preds

In [ ]:
stripped_preds = []
for i in preds:
  s = i.replace("<pad>", "")
  s = s.replace("</s>", "")
  s = s.replace("en_XX", "")
  stripped_preds.append(s)

In [ ]:
stripped_preds

In [ ]:
stripped_preds[2], x_test[2]

In [ ]:
import pickle
with open("stripped_preds_3.pkl", "wb") as f:
    # Use the pickle.dump() method to pickle the data to the file
    pickle.dump(stripped_preds, f)

In [ ]:
!pip install sacrebleu

In [ ]:
# Example reference translations
references = [stripped_preds[2]]

# Example translations to be evaluated
translations = [x_test[2]]

import sacrebleu

# Calculate the SacreBLEU score
bleu = sacrebleu.corpus_bleu(translations, [references])

print("SacreBLEU score:", bleu.score)

In [ ]:
bleu_list = []
for i in range(len(stripped_preds)):
  references = [stripped_preds[i]]
  translations = [x_test[i]]
  bleu = sacrebleu.corpus_bleu(translations, [references])
  bleu_list.append(bleu.score)

In [ ]:
bleu_list

In [ ]:
np.average(bleu_list)

In [ ]:
stripped_preds[0]

In [ ]:
x_test[0]

In [ ]:
trainer.save_model("/content/drive/MyDrive/my_trained_model")

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/my_trained_model")

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/my_trained_model")

In [ ]:
trainer_loaded = Seq2SeqTrainer(model=model, args=Seq2SeqTrainingArguments( output_dir="/content/drive/MyDrive"))

In [ ]:
model

In [ ]:
model.state_dict()

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/trained_model_mbart50.pth')

In [ ]:
model2 = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model2.load_state_dict(torch.load('/content/drive/MyDrive/trained_model_mbart50.pth'))

In [ ]:
model2